In [1]:
import lib.custom_callbacks as callbacks
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-TRAIN-80.csv'
VALIDATION_FILE = 'DATASET-VALIDATION-10_BALANCED.csv'

ONE_HOT_FORMAT = False

TRAINING_BATCH_SIZE = 4
TRAINING_DATA_AUG = True
TRAINING_BALANCED = True
TRAINING_PREFETCH = 20
TRAINING_TAKE_SIZE = None

VALIDATION_BATCH_SIZE = 12
VALIDATION_DATA_AUG = False
VALIDATION_BALANCED = False
VALIDATION_PREFETCH = 1
VALIDATION_TAKE_SIZE = None

In [4]:
reload(dt)

'''
def create_dataset_new(csv_file, 
                        list_list_classes,
                        balanced=False,
                        apply_data_augmentation=False,
                        batch_size=1, 
                        prefetch_buffer=None, 
                        shuffle=False,
                        size=None):
'''

train_dataset, y_true_train = dt.create_dataset_new(TRAIN_FILE, 
                                                    DR_LEVELS_PER_CLASS, 
                                                    balanced=TRAINING_BALANCED, 
                                                    apply_data_augmentation=TRAINING_DATA_AUG, 
                                                    batch_size=TRAINING_BATCH_SIZE, 
                                                    prefetch_buffer=TRAINING_PREFETCH, 
                                                    one_hot_format=ONE_HOT_FORMAT,
                                                    size=TRAINING_TAKE_SIZE)

val_dataset, y_true_val = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                is_validation=True,
                                                one_hot_format=ONE_HOT_FORMAT,
                                                size=VALIDATION_TAKE_SIZE)

Checking num images per label...
Class/label 0 has 42149 images
Class/label 1 has 14673 images
Apply shuffle on each one-label dataset
Apply shuffle on each one-label dataset
This balanced dataset will have 29346 images, having 14673 images per class
One-label datasets joined into a one single tf.data.Dataset object
Shuffle all dataset
Se aplica data aug


In [5]:
# Define model
reload(models)

model = models.resNet50V2.get_model(input_shape=(540,540,3), num_outputs=len(DR_LEVELS_PER_CLASS))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 2048)              23564800  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,568,898
Trainable params: 23,523,458
Non-trainable params: 45,440
_________________________________________________________________


In [6]:
# Load weights if they were saved
base_path = 'saved_weights/resNet50V2/RMSProp_bal_bs4/'
save_path = base_path + 'model.h5'

base_path_splitted = base_path.split('/')
for i in range(len(base_path_splitted)):
    path_i = '/'.join(base_path_splitted[:i+1])
    if not os.path.exists(path_i):
        os.mkdir(path_i)

if os.path.exists(save_path):
    model.load_weights(save_path)
    print('Model loaded')

In [7]:
# Save ground truth for validation dataset
np.save(base_path + 'ground_truth_val.npy', y_true_val)

In [8]:
# Compile model
reload(metrics)

classes_names = [''.join(list(map(str,class_))) for class_ in DR_LEVELS_PER_CLASS]

print(classes_names)

metric_AUC_0_1234 = metrics.Wrapper_AUC(classes=[1], original_dr_lvls=classes_names, is_one_hot=ONE_HOT_FORMAT)

metric_Sp_at_95_Sens_0_1234 = metrics.Wrapper_SpecificityAtSensitivity(sensitivity=0.95, classes=[1], original_dr_lvls=classes_names, is_one_hot=ONE_HOT_FORMAT)

['0', '1234']


In [9]:
cbacks = [tf.keras.callbacks.ModelCheckpoint(base_path + 'best' + metric_AUC_0_1234.get_custom_name() + '.h5', 
                                             monitor='val_' + metric_AUC_0_1234.get_custom_name(),
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),

          tf.keras.callbacks.ModelCheckpoint(base_path + 'best' + metric_Sp_at_95_Sens_0_1234.get_custom_name() + '.h5', 
                                             monitor='val_' + metric_Sp_at_95_Sens_0_1234.get_custom_name(),
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),
          
          tf.keras.callbacks.ModelCheckpoint(base_path + 'best_accuracy.h5',
                                             monitor='val_accuracy',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),

          tf.keras.callbacks.ModelCheckpoint(base_path + 'best_loss.h5',
                                             monitor='val_loss',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='min'),
          
          callbacks.Save_Training_Evolution(base_path + 'training_evolution.csv')
]

# cbacks = [tf.keras.callbacks.ModelCheckpoint(base_path + 'best_loss.h5',
#                                              monitor='val_loss',
#                                              save_best_only=True,
#                                              save_weights_only=True,
#                                              mode='min'),
          
#           callbacks.Save_Training_Evolution(base_path + 'training_evolution.csv')
# ]

In [10]:
reload(metrics)

validation_dir = base_path+'validation_outputs/'

if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

'''
From Voets' GitHub (Google replication 2018-19)
# This is TF 1.X
decay = 4e-5 # This value is the weight decay specified in their papers
...
train_op = tf.train.RMSPropOptimizer(
    learning_rate=learning_rate, decay=decay) \ # so, in TF 1.x, RMSProp has an argument called 'decay' for this purpose
        .minimize(loss=mean_xentropy, global_step=global_step)

### TF 1.x RMSProp description
https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/RMSPropOptimizer

Argument 'decay' -- Discounting factor for the history/coming gradient

### TF 2.x RMSProp description
https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/RMSprop

Argument 'rho' -- Discounting factor for the history/coming gradient. Defaults to 0.9.

So, 'rho' should be the 'weight decay' argument specified on Voets' code
'''
    
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=4e-5, clipnorm=1.0), # There is no momentum reference in the papers
              metrics=['accuracy',
                       metric_AUC_0_1234, # DR levels: 0 vs 1,2,3,4
                       metric_Sp_at_95_Sens_0_1234, # DR levels: 0 vs 1,2,3,4
                       metrics.RunningValidation(path=validation_dir, n_columns=len(DR_LEVELS_PER_CLASS)) # THIS METRIC MUST BE INIZIALIZATED BEFORE EVERY TRAINING
                      ]) 

num_epochs = 2000

# history = model.fit(train_dataset.take(50), epochs=num_epochs, validation_data=val_dataset.take(10), verbose=1, callbacks=cbacks, class_weight=d_class_weights)
history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=1, callbacks=cbacks)

Epoch 1/2000
7337/7337 [==============================] - 832s 113ms/step - loss: 0.8873 - accuracy: 0.5009 - AUC_DRlvls_0_1234: 0.4921 - Sp_at_95_sens_DRlvls_0_1234: 0.0558 - RunningValidation: 0.0000e+00 - val_loss: 0.6941 - val_accuracy: 0.5111 - val_AUC_DRlvls_0_1234: 0.0726 - val_Sp_at_95_sens_DRlvls_0_1234: 0.0439 - val_RunningValidation: 1.0000
Epoch 2/2000
7337/7337 [==============================] - 843s 115ms/step - loss: 0.8323 - accuracy: 0.5006 - AUC_DRlvls_0_1234: 0.4875 - Sp_at_95_sens_DRlvls_0_1234: 0.0495 - RunningValidation: 0.0000e+00 - val_loss: 0.6979 - val_accuracy: 0.5172 - val_AUC_DRlvls_0_1234: 0.4699 - val_Sp_at_95_sens_DRlvls_0_1234: 0.0384 - val_RunningValidation: 1.0000
Epoch 3/2000
7337/7337 [==============================] - 855s 117ms/step - loss: 0.7804 - accuracy: 0.5115 - AUC_DRlvls_0_1234: 0.5049 - Sp_at_95_sens_DRlvls_0_1234: 0.0534 - RunningValidation: 0.0000e+00 - val_loss: 0.7448 - val_accuracy: 0.5125 - val_AUC_DRlvls_0_1234: 0.5272 - val_Sp_at_

KeyboardInterrupt: 